In [1]:
#!/usr/bin/env python3
"""
Send a GET to https://api.ipify.org?format=json
once from every locally‑bound IP address in `MY_IPS`.
Each request is forced to leave the machine with that
particular source IP, and the User‑Agent / Accept headers
are shuffled for a little header variability.
Tested with Python 3.11 on Ubuntu 24.04.
"""

import random
import time
import requests
from requests.adapters import HTTPAdapter
from urllib3.util import Retry

# --------------------------------------------------------------------------- #
# 1.  List every secondary address that is configured on the host.            #
#    Ensure that `ip addr` shows them first.                                  #
# --------------------------------------------------------------------------- #
MY_IPS = [
    "79.175.189.77",
    "46.102.143.181",
    "46.102.132.34",
    "93.113.233.105",
    "185.3.125.197",
    "79.175.189.125",
]

# --------------------------------------------------------------------------- #
# 2.  A few plausible User‑Agent and Accept‑Language strings to randomise.    #
# --------------------------------------------------------------------------- #
USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
    "(KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36",
    "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:128.0) Gecko/20100101 Firefox/128.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 14_5_0) AppleWebKit/605.1.15 "
    "(KHTML, like Gecko) Version/17.5 Safari/605.1.15",
]

ACCEPT_LANGS = [
    "en-US,en;q=0.9",
    "en-GB,en;q=0.8,de;q=0.6",
    "fr-FR,fr;q=0.8,en;q=0.6",
]

# --------------------------------------------------------------------------- #
# 3.  An adapter that binds the underlying socket to a chosen source address. #
#    Works for both HTTP(S) and keeps urllib3’s connection pooling intact.    #
# --------------------------------------------------------------------------- #
class SourceIPAdapter(HTTPAdapter):
    """Bind outbound sockets to a user‑supplied source IP."""

    def __init__(self, source_ip: str, **kwargs):
        self._source_addr = (source_ip, 0)
        super().__init__(max_retries=Retry(total=2, backoff_factor=0.3), **kwargs)

    # Called for regular (non‑proxy) connections
    def init_poolmanager(self, *args, **kwargs):
        kwargs["source_address"] = self._source_addr
        return super().init_poolmanager(*args, **kwargs)

    # Called when a proxy is in use (not the case here, but keep consistent)
    def proxy_manager_for(self, *args, **kwargs):
        kwargs["source_address"] = self._source_addr
        return super().proxy_manager_for(*args, **kwargs)

# --------------------------------------------------------------------------- #
# 4.  One helper that sends a single request from one source IP.              #
# --------------------------------------------------------------------------- #
TARGET_URL = "https://api.ipify.org?format=json"

def hit_ipify(source_ip: str) -> None:
    """Ask api.ipify.org from a given local address and print what it sees."""
    session = requests.Session()

    # Mount the adapter for both HTTP and HTTPS schemes
    adapter = SourceIPAdapter(source_ip)
    session.mount("http://", adapter)
    session.mount("https://", adapter)

    # Rotate headers
    session.headers.update(
        {
            "User-Agent": random.choice(USER_AGENTS),
            "Accept-Language": random.choice(ACCEPT_LANGS),
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
        }
    )

    try:
        resp = session.get(TARGET_URL, timeout=8)
        resp.raise_for_status()
        remote_view = resp.json().get("ip", "N/A")
        print(f"Asked via {source_ip:<15}  ➜  ipify reports {remote_view}")
    except Exception as exc:
        print(f"[{source_ip}] request failed: {exc}")
    finally:
        session.close()

# --------------------------------------------------------------------------- #
# 5.  Drive the whole thing (sequentially here; swap for threading/asyncio    #
#     if throughput matters).                                                #
# --------------------------------------------------------------------------- #
if __name__ == "__main__":
    random.shuffle(MY_IPS)                 # touch every address, random order
    for ip in MY_IPS:
        hit_ipify(ip)
        time.sleep(random.uniform(0.3, 1.0))   # polite delay


Asked via 185.3.125.197    ➜  ipify reports 185.3.125.197
Asked via 79.175.189.125   ➜  ipify reports 79.175.189.125
Asked via 79.175.189.77    ➜  ipify reports 79.175.189.77
Asked via 93.113.233.105   ➜  ipify reports 93.113.233.105
Asked via 46.102.132.34    ➜  ipify reports 46.102.132.34
Asked via 46.102.143.181   ➜  ipify reports 46.102.143.181
